In [ ]:
import cv2
from glob import glob
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
images = []
labels = []

def get_data(clazz):
    files = glob(os.path.join('..', 'input','state-farm-distracted-driver-detection','imgs', 'train', 'c' + str(clazz), '*.jpg'))
    for file in files:
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        images.append(cv2.resize(img, (225,225)))
        labels.append(clazz)

In [ ]:
for clazz in range(10):
    get_data(clazz)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=0)

In [ ]:
from keras.utils import to_categorical
import numpy as np

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

x_test = np.expand_dims(x_test, axis=3)
x_train = np.expand_dims(x_train, axis=3)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
fig = plt.figure(figsize=(8,8))
for i in range(64):
    ax = fig.add_subplot(8,8,i+1)
    ax.imshow(x_train[i], cmap=plt.cm.bone)
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, MaxPooling2D, Dropout, Flatten
from keras.callbacks import EarlyStopping

model = Sequential()

model.add(Conv2D(32 ,kernel_size=(3, 3), padding='same', input_shape=(225, 225, 1), strides=(1, 1), activation='elu'))
model.add(Conv2D(32, (3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64 ,kernel_size=(3, 3), padding='same', activation='elu'))
model.add(Conv2D(64, (3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128 ,kernel_size=(3, 3), padding='same', activation='elu'))
model.add(Conv2D(128, (3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='elu'))
model.add(Dropout(0.25))

model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import RMSprop

opt = RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=50)

In [ ]:
_, acc = model.evaluate(x_test, y_test, verbose=10)
print(acc)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
np.savetxt(fname="submission.csv", X=predictions)